In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import pickle

from sklearn.preprocessing import StandardScaler

In [ ]:
data = pd.read_csv('Stocks.csv')
data

In [ ]:
data.plot()

In [ ]:
class MultiStockEnv:
    """
    A 3-stock trading environment.
    State: vector of size 7 (n_stock * 2 + 1)
        - # shares of stock 1 owned
        - # shares of stock 2 owned
        - # shares of stock 3 owned
        - price of stock 1 (using daily close price)
        - price of stock 2
        - price of stock 3
        - cash owned (can be used to purchase more stocks)
    Action: categorical variable with 27 (3^3) possibilities
        - for each stock, you can:
        - 0 = sell
        - 1 = hold
        - 2 = buy
    """
    def __init__(self, data, initial_investment=20000):
        # data
        self.stock_price_history = data
        self.n_step, self.n_stock = self.stock_price_history.shape

        # instance attributes
        self.initial_investment = initial_investment
        self.cur_step = None
        self.stock_owned = None
        self.stock_price = None
        self.cash_in_hand = None

        self.action_space = np.arange(3**self.n_stock)

        # action permutations
        # returns a nested list with elements like:
        # [0,0,0]
        # [0,0,1]
        # [0,0,2]
        # [0,1,0]
        # [0,1,1]
        # etc.
        # 0 = sell
        # 1 = hold
        # 2 = buy
        self.action_list = list(map(list, itertools.product([0, 1, 2], repeat=self.n_stock)))

        # calculate size of state
        self.state_dim = self.n_stock * 2 + 1

        self.reset()


    def reset(self):
        self.cur_step = 0
        self.stock_owned = np.zeros(self.n_stock)
        self.stock_price = self.stock_price_history[self.cur_step]
        self.cash_in_hand = self.initial_investment
        return self._get_obs()


    def step(self, action):
        assert action in self.action_space

        # get current value before performing the action
        prev_val = self._get_val()

        # update price, i.e. go to the next day
        self.cur_step += 1
        self.stock_price = self.stock_price_history[self.cur_step]

        # perform the trade
        self._trade(action)

        # get the new value after taking the action
        cur_val = self._get_val()

        # reward is the increase in porfolio value
        reward = cur_val - prev_val

        # done if we have run out of data
        done = self.cur_step == self.n_step - 1

        # store the current value of the portfolio here
        info = {'cur_val': cur_val}

        # conform to the Gym API
        return self._get_obs(), reward, done, info


    def _get_obs(self):
        obs = np.empty(self.state_dim)
        obs[:self.n_stock] = self.stock_owned
        obs[self.n_stock:2*self.n_stock] = self.stock_price
        obs[-1] = self.cash_in_hand
        return obs
    
    def _get_val(self):
        return self.stock_owned.dot(self.stock_price) + self.cash_in_hand

    def _trade(self, action):
        # index the action we want to perform
        # 0 = sell
        # 1 = hold
        # 2 = buy
        # e.g. [2,1,0] means:
        # buy first stock
        # hold second stock
        # sell third stock
        action_vec = self.action_list[action]

        # determine which stocks to buy or sell
        sell_index = [] # stores index of stocks we want to sell
        buy_index = [] # stores index of stocks we want to buy
        for i, a in enumerate(action_vec):
            if a == 0:
                sell_index.append(i)
            elif a == 2:
                buy_index.append(i)

        # sell any stocks we want to sell
        # then buy any stocks we want to buy
        if sell_index:
            # NOTE: to simplify the problem, when we sell, we will sell ALL shares of that stock
            for i in sell_index:
                self.cash_in_hand += self.stock_price[i] * self.stock_owned[i]
                self.stock_owned[i] = 0
        if buy_index:
            # NOTE: when buying, we will loop through each stock we want to buy,
            #       and buy one share at a time until we run out of cash
            can_buy = True
            while can_buy:
                for i in buy_index:
                    if self.cash_in_hand > self.stock_price[i]:
                        self.stock_owned[i] += 1 # buy one share
                        self.cash_in_hand -= self.stock_price[i]
                    else:
                        can_buy = False

In [ ]:
action_list = list(map(list, itertools.product([0, 1, 2], repeat=3)))

In [ ]:
action_list

In [ ]:
n_timesteps, n_stocks = data.shape

In [ ]:
n_train = n_timesteps // 2
n_train

In [ ]:
train_data = data[:n_train].values
test_data = data[n_train:].values

In [ ]:
env = MultiStockEnv(train_data,initial_investment=20000)

In [ ]:
env.reset()

In [ ]:
env.action_list

In [ ]:
env.step(26)

In [ ]:
def predict(X,W,b):
    # make sure X is N x D
    assert(len(X.shape) == 2)
    return X.dot(W) + b

In [ ]:
def sgd(X,Y,W,b,vW,vb,learning_rate=0.01, momentum=0.9):
    assert(len(X.shape) == 2)
    # the loss values are 2-D
    # normally we would divide by N only
    # but now we divide by N x K
    num_values = np.prod(Y.shape)
    # do one step of gradient descent
    # we multiply by 2 to get the exact gradient
    # (not adjusting the learning rate)
    # i.e. d/dx (x^2) --> 2x
    Yhat = predict(X,W,b)
    gW = 2 * X.T.dot(Yhat - Y) / num_values
    gb = 2 * (Yhat - Y).sum(axis=0) / num_values

    # update momentum terms
    vW = momentum * vW - learning_rate * gW
    vb = momentum * vb - learning_rate * gb

    # update params
    W += vW
    b += vb
    return W,b,vW,vb

In [ ]:
def greedy_policy(state,W,b,epsilon):
    if np.random.rand() <= epsilon:
        return np.random.choice(action_size)
    else:
        act_values = predict(state,W,b)
        return np.argmax(act_values[0])  # returns action

In [ ]:
def get_scaler(env):
    # return scikit-learn scaler object to scale the states after fitting the scaler with some random actions
    states = []
    #play game 100 times with random actions to collect some states
    for i in range(100):
        for j in range(env.n_step):
            action = np.random.choice(env.action_space)
            state, reward, done, info = env.step(action)
            states.append(state)
            if done:
                env.reset()
                break
    scaler = StandardScaler()
    scaler.fit(states)
    return scaler

In [ ]:
gamma = 0.95  # discount rate
epsilon = 1.0  # exploration rate
epsilon_min = 0.01
epsilon_decay = 0.995
num_episodes = 2000
initial_investment = 20000

n_timesteps, n_stocks = data.shape
n_train = n_timesteps // 2
train_data = data[:n_train].values
test_data = data[n_train:].values

np.random.seed(1)
env = MultiStockEnv(train_data,initial_investment=20000)
#Train the scaler, the scaler only uses the training set
state_size = env.state_dim
action_size = len(env.action_space)

#Initialize weights
W = np.random.randn(state_size, action_size) / np.sqrt(state_size)
b = np.zeros(action_size)
scaler=get_scaler(env)

#Initialize momentum terms
vW = 0
vb = 0

# store the final value of the portfolio (end of episode)
portfolio_value = []

for i in range(num_episodes):
    done=False
    state = env.reset()
    state = scaler.transform([state])
    while not done:
        action = greedy_policy(state,W,b,epsilon)
        next_state, reward, done, info = env.step(action)
        next_state = scaler.transform([next_state])
        if done:
            target = reward
        else:
            target = reward + gamma * np.amax(predict(next_state,W,b), axis=1)
        #Make prediction for all actions first
        target_full = predict(state,W,b)
        #Update target values for only the selected action 
        target_full[0, action] = target
        #Update weights
        W,b,vW,vb=sgd(X=state, Y=target_full,W=W,b=b,vW=vW,vb=vb)
        
        state=next_state
        
        if epsilon > epsilon_min:
            epsilon *= epsilon_decay
        
    val=info['cur_val']
    print(f"episode: {i + 1}/{num_episodes}, episode end value: {val:.2f}")
    portfolio_value.append(val)
pickle.dump(scaler, open('scaler.pkl','wb'))
np.savez("TraderWeights.npz", W=W, b=b)

In [ ]:
scaler = pickle.load(open('scaler.pkl', 'rb'))
npz = np.load("TraderWeights.npz")
W = npz['W']
b = npz['b']

In [ ]:
W

In [ ]:
np.random.seed(1)
#Test the performance of the agent with the test data
epsilon = 0.01
env = MultiStockEnv(test_data, initial_investment)
testportfoliovalues=[]
for i in range(num_episodes):
    state = env.reset()
    state = scaler.transform([state])
    done=False
    while not done:
        action = greedy_policy(state,W,b,epsilon=epsilon)
        next_state, reward, done, info = env.step(action)
        next_state = scaler.transform([next_state])
        state = next_state
    value=info['cur_val']
    print(f"episode: {i + 1}/{num_episodes}, episode end value: {value:.2f}")
    testportfoliovalues.append(value) # append episode end portfolio value

In [ ]:
np.random.seed(1)
#How does a random policy do?
epsilon = 1
env = MultiStockEnv(test_data, initial_investment)
randomportfoliovalues=[]
for i in range(num_episodes):
    state = env.reset()
    state = scaler.transform([state])
    done=False
    while not done:
        action = greedy_policy(state,W,b,epsilon=epsilon)
        next_state, reward, done, info = env.step(action)
        next_state = scaler.transform([next_state])
        state = next_state
    value=info['cur_val']
    print(f"episode: {i + 1}/{num_episodes}, episode end value: {value:.2f}")
    randomportfoliovalues.append(value) # append episode end portfolio value

In [ ]:
np.savez("TraderPortfolios.npz", TrainingPortfolio=portfolio_value, 
         TestPortfolio=testportfoliovalues,
         RandomPortfolio=randomportfoliovalues)

In [ ]:
npz = np.load("TraderPortfolios.npz")
TrainingPortfolio=npz['TrainingPortfolio']
RandomPortfolio=npz['RandomPortfolio']
TestPortfolio=npz['TestPortfolio']

In [ ]:
plt.plot(TrainingPortfolio)

In [ ]:
plt.hist(TrainingPortfolio)

In [ ]:
np.mean(TestPortfolio)

In [ ]:
plt.hist(TestPortfolio)

In [ ]:
plt.hist(RandomPortfolio)

In [ ]:
np.mean(RandomPortfolio)

In [ ]:
test_data[-1]

In [ ]:
PercentIncrease=(test_data[-1]-test_data[0])/test_data[0]

In [ ]:
np.mean(PercentIncrease)